In [4]:
import numpy as np

## Pseudo code

```
function build_tree(data, labels, depth=0):
    if stop_condition(data, labels, depth):
        return LeafNode(class = majority_class(labels))
    
    best_gain = 0
    best_feat, best_thresh = None, None
    parent_impurity = impurity(labels)

    for each feature j in 1…D:
        for each threshold t in unique_values(data[:,j]):
            left_labels  = labels[data[:,j] ≤ t]
            right_labels = labels[data[:,j] >  t]

            if len(left_labels)==0 or len(right_labels)==0: continue

            gain = parent_impurity \
                   - (|left|/|total|)*impurity(left_labels) \
                   - (|right|/|total|)*impurity(right_labels)

            if gain > best_gain:
                best_gain, best_feat, best_thresh = gain, j, t

    if best_gain < min_impurity_decrease:
        return LeafNode(class = majority_class(labels))

    left_data, left_labels  = split(data, labels, best_feat, best_thresh, side="left")
    right_data, right_labels = split(data, labels, best_feat, best_thresh, side="right")

    left_subtree  = build_tree(left_data,  left_labels,  depth+1)
    right_subtree = build_tree(right_data, right_labels, depth+1)

    return DecisionNode(
        feature_index = best_feat,
        threshold     = best_thresh,
        left          = left_subtree,
        right         = right_subtree
    )

```

In [5]:
import pandas as pd
pd.set_option('display.max_columns', 100)
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

In [6]:
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
y[:5]

array([0, 0, 0, 0, 0])

In [8]:
X.shape

(569, 30)

In [9]:
pd.DataFrame(y).value_counts(), Counter(y).most_common(1)[0][0], np.bincount(y)

(0
 1    357
 0    212
 Name: count, dtype: int64,
 np.int64(1),
 array([212, 357]))

In [10]:
(X[:, 0] < 15).sum()

np.int64(395)

In [13]:
import numpy as np
from collections import Counter

class DecisionNode:
    def __init__(self, feature_index, threshold, left, right):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right

class LeafNode:
    def __init__(self, prediction):
        self.prediction = prediction

class CustomDecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2,
                 min_impurity_decrease=1e-7, criterion="gini"):
        assert criterion in ("gini", "entropy", "misclassification"), \
            "criterion must be 'gini', 'entropy', or 'misclassification'"
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.criterion = criterion
        self.n_classes = None
        self.root = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.n_classes = len(set(y))
        self.root = self._build_tree(X, y, depth=0)

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

    def print_tree(self):
        self._print_node(self.root, spacing="")

    def _print_node(self, node, spacing):
        if isinstance(node, LeafNode):
            print(spacing + f"Predict → {node.prediction}")
            return
        print(spacing + f"[X[{node.feature_index}] ≤ {node.threshold:.4f}]")
        print(spacing + " ➜ True:")
        self._print_node(node.left, spacing + "    ")
        print(spacing + " ➜ False:")
        self._print_node(node.right, spacing + "    ")

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        # stopping criteria
        if (depth == self.max_depth or
            num_samples < self.min_samples_split or
            len(set(y)) == 1):
            leaf_label = Counter(y).most_common(1)[0][0]
            return LeafNode(leaf_label)

        parent_impurity = self._impurity(y)
        best_gain = 0.0
        best_feat = None
        best_thresh = None

        # find best split
        for feat in range(num_features):
            thresholds = np.unique(X[:, feat])
            for thresh in thresholds:
                left_mask = X[:, feat] <= thresh
                right_mask = ~left_mask
                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue
                gain = self._information_gain(y, left_mask, right_mask, parent_impurity)
                if gain > best_gain:
                    best_gain = gain
                    best_feat = feat
                    best_thresh = thresh

        # no valid split found
        if best_gain < self.min_impurity_decrease:
            leaf_label = Counter(y).most_common(1)[0][0]
            return LeafNode(leaf_label)

        # split on best feature/threshold
        mask_left = X[:, best_feat] <= best_thresh
        mask_right = ~mask_left
        left_subtree = self._build_tree(X[mask_left], y[mask_left], depth+1)
        right_subtree = self._build_tree(X[mask_right], y[mask_right], depth+1)
        return DecisionNode(best_feat, best_thresh, left_subtree, right_subtree)

    def _impurity(self, y):
        counts = np.bincount(y, minlength=self.n_classes)
        ps = counts / counts.sum()
        if self.criterion == "gini":
            return 1 - np.sum(ps**2)
        elif self.criterion == "entropy":
            return -np.sum([p * np.log2(p) for p in ps if p > 0])
        else:  # misclassification error
            return 1 - np.max(ps)

    def _information_gain(self, y, left_mask, right_mask, parent_impurity):
        n = len(y)
        n_left = left_mask.sum()
        n_right = right_mask.sum()
        imp_left = self._impurity(y[left_mask])
        imp_right = self._impurity(y[right_mask])
        child_impurity = (n_left/n)*imp_left + (n_right/n)*imp_right
        return parent_impurity - child_impurity

    def _predict_one(self, x, node):
        if isinstance(node, LeafNode):
            return node.prediction
        if x[node.feature_index] <= node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

In [14]:
tree = CustomDecisionTreeClassifier(max_depth=5, min_samples_split=2, criterion="gini")
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print("Accuracy:", accuracy(y_test, y_pred))

# Check if the implementation is correct
from sklearn.tree import DecisionTreeClassifier
sk_tree = DecisionTreeClassifier(max_depth=5, min_samples_split=2, criterion="gini")
sk_tree.fit(X_train, y_train)
y_sk_pred = sk_tree.predict(X_test)

print("Sklearn Accuracy:", accuracy(y_test, y_sk_pred))

Accuracy: 0.9035087719298246
Sklearn Accuracy: 0.9122807017543859
